In [33]:
import pyvista as pv
import numpy as np
import pandas as pd
from pathlib import Path

def remove_open_bracket(df):
    # Remove '(' character from every value
    df = df.applymap(lambda x: x.replace('(', ''))

#     print(df)
    return df

def remove_empty_strings(lst):
    return list(filter(lambda x: x != '', lst))


# Split by ' ' for each element in DataFrame
def split_by_space(df):
    df=df.applymap(lambda x: x.split(' '))
    
    return df

In [49]:
def load_post_processed_data(path2evectors):
    
    # Load original data
    df=pd.read_csv(path, names=['v1', 'v2', 'v3'], skiprows=3, sep=')', index_col=False)
    print(df.shape)
    
    # Remove openbracket from all elements
    df=remove_open_bracket(df)    
    print(df.shape)

    # Split all elements by space
    df=split_by_space(df)
    print(df.shape)

    # Remove empty strings in lists in elements
    df=df.applymap(remove_empty_strings)
    print(df.shape)

    # Unpack DataFrame into array
    ar=df.values
    print(ar.shape)

    ar_fl=ar.flatten()
    print(ar_fl.shape)
    
    # Reshape array
    unpacked_array = np.array([[float(x) for x in sublist] for sublist in ar_fl])
    print(unpacked_array.shape)

    # reshape to final desired shape N_cells X 3 X 3

    final_ar = unpacked_array.reshape((df.shape[0], 3, 3))
    print(final_ar.shape)
    
#     print(final_ar)
    
    return final_ar

In [50]:
evectors = load_post_processed_data(path2evectors=path)

(289521, 3)
(289521, 3)
(289521, 3)
(289521, 3)
(289521, 3)
(868563,)
(868563, 3)
(289521, 3, 3)


# Load Fibres Directions

In [43]:
path2fibres=casePath+'/dcm0/Fibres_HaoSeg'+'/Labelled_Endo_avg.lon'

In [45]:
fib_dirs=np.loadtxt(path2fibres)

In [46]:
fib_dirs.shape

(289521, 3)

# Load Original Eigenvalue Data

In [54]:
evaluePath=f"{path_pl.parent}/strain-eigenvalues-t4.txt"

In [69]:
df=pd.read_csv(evaluePath, skiprows=3, names=['e1'])

In [70]:
df=split_by_space(df)
df=df.applymap(remove_empty_strings)

In [76]:
# Unpack DataFrame into array
ar=df.values
print(ar.shape)

ar_fl=ar.flatten()
print(ar_fl.shape)

(289521, 1)
(289521,)


In [77]:
# Reshape array
unpacked_array = np.array([[float(x) for x in sublist] for sublist in ar_fl])
print(unpacked_array.shape)

# reshape to final desired shape N_cells X 3 X 3

final_ar = unpacked_array.reshape((df.shape[0], 3))
print(final_ar.shape)

(289521, 3)
(289521, 3)


In [78]:
final_ar

array([[ 2.58441e-01, -1.36040e-01,  3.47408e-10],
       [ 2.80601e-01, -9.25817e-02,  7.19853e-10],
       [ 2.65797e-01, -1.77578e-01,  7.37359e-10],
       ...,
       [-5.64121e-09,  4.36079e-02,  2.23319e-01],
       [ 6.74711e-02, -8.03650e-10,  2.19923e-01],
       [ 2.90418e-09,  4.89952e-02,  1.70844e-01]])

In [79]:
df

,e1
0,"[0.258441, -0.13604, 3.47408e-10]"
1,"[0.280601, -0.0925817, 7.19853e-10]"
2,"[0.265797, -0.177578, 7.37359e-10]"
3,"[0.0748082, 3.38454e-10, -0.0847964]"
4,"[0.0548211, -6.29629e-09, -0.102557]"
...,...
289516,"[-4.51252e-09, 0.0333084, 0.237808]"
289517,"[-1.02411e-08, 0.0587583, 0.258005]"
289518,"[-5.64121e-09, 0.0436079, 0.223319]"
289519,"[0.0674711, -8.0365e-10, 0.219923]"


In [108]:
max_indices = np.argmax(np.abs(final_ar), axis=1)

In [109]:
max_indices

array([0, 0, 0, ..., 2, 2, 2])

In [110]:
max_indices.shape

(289521,)

In [111]:
evectors.shape

(289521, 3, 3)

In [113]:
max_indices[:8]

array([0, 0, 0, 2, 2, 0, 0, 0])

In [114]:
np.round(final_ar[:8, :], 4)

array([[ 0.2584, -0.136 ,  0.    ],
       [ 0.2806, -0.0926,  0.    ],
       [ 0.2658, -0.1776,  0.    ],
       [ 0.0748,  0.    , -0.0848],
       [ 0.0548, -0.    , -0.1026],
       [ 0.2687, -0.1791, -0.    ],
       [ 0.3292, -0.1768, -0.    ],
       [ 0.3171, -0.1677,  0.    ]])

In [116]:
fib_dirs.shape

(289521, 3)

In [119]:
# do the dot product between relevant evector component with fibres direction:

dot_results = np.zeros((df.shape[0],))

for i in range(evectors.shape[0]):
    max_idx = max_indices[i]
    
    princ_dir = evectors[i, max_idx, :]
    
    ans = np.dot(princ_dir, fib_dirs[i, :])
#     print(ans)
    dot_results[i]=ans

In [125]:
dot_results_abs=np.abs(dot_results)
dot_results_abs.mean()

0.6042570714771016

# To test:

* Are fibre directions more aligned with principle strains than, say, randomly assigned fibre directions?
* Difference for phases:
    * Reservoir (passive)
    * Contractile (active)
        * Try first principle strains wrt dcm0?
        * Need to do wrt reservoir geometry?

# Load Original Eigenvector Data

In [15]:
# load original eigen vectors.txt path

casePath="/media/csi20local/One Touch/Backup_April2023/Dropbox/phd/Data/RG_CT_Cases/CT-CRT/case01"
path=f"{casePath}/MT-HiRes/SW-0.0-BE-1e-9/strain-eigenvectors-t4.txt"
path_pl=Path(path)

df=pd.read_csv(path, names=['v1', 'v2', 'v3'], skiprows=3, sep=')', index_col=False)

In [16]:
## print original df
print(df.shape)
df.head()

(289521, 3)


,v1,v2,v3
0,( 0.731327 -0.586307 0.348433,(-0.676398 -0.688998 0.260321,(-0.0874416 0.426059 0.90046
1,(-0.808864 0.418552 -0.41298,(-0.531938 -0.820171 0.210619,( 0.250559 -0.390043 -0.886051
2,( 0.68376 -0.617386 0.388981,(-0.729447 -0.564085 0.38693,(0.0194665 0.548309 0.83605
3,(-0.770767 -0.127292 -0.624272,( -0.627478 -0.0181393 0.778423,( 0.110411 -0.991699 0.0658919
4,(0.693495 0.2807 0.66353,(-0.655229 -0.137211 0.742865,(-0.299566 0.949937 -0.088768


## Remove OpenBracket from all elements

In [17]:
df=remove_open_bracket(df)

In [18]:
print(df.shape)
df.head()

(289521, 3)


,v1,v2,v3
0,0.731327 -0.586307 0.348433,-0.676398 -0.688998 0.260321,-0.0874416 0.426059 0.90046
1,-0.808864 0.418552 -0.41298,-0.531938 -0.820171 0.210619,0.250559 -0.390043 -0.886051
2,0.68376 -0.617386 0.388981,-0.729447 -0.564085 0.38693,0.0194665 0.548309 0.83605
3,-0.770767 -0.127292 -0.624272,-0.627478 -0.0181393 0.778423,0.110411 -0.991699 0.0658919
4,0.693495 0.2807 0.66353,-0.655229 -0.137211 0.742865,-0.299566 0.949937 -0.088768


In [ ]:
list_ = df.iloc[0, 0].split(' ')

# Remove empty entries using a list comprehension
filtered_list = [x for x in list_ if x != ""]

print(filtered_list)


df.shape

for i in range(df.shape[0]):
    for j in range(df.shape[1]):
        
        list_ = df.iloc[i, j].split(' ')
        # Remove empty entries using a list comprehension
        filtered_list = [x for x in list_ if x != ""]
        print(filtered_list)


In [20]:
df.values

array([[' 0.731327 -0.586307  0.348433',
        ' -0.676398 -0.688998  0.260321',
        ' -0.0874416   0.426059    0.90046'],
       ['-0.808864  0.418552  -0.41298',
        ' -0.531938 -0.820171  0.210619',
        '  0.250559 -0.390043 -0.886051'],
       ['  0.68376 -0.617386  0.388981',
        ' -0.729447 -0.564085   0.38693',
        ' 0.0194665  0.548309   0.83605'],
       ...,
       [' 0.737923 -0.592197 -0.323687',
        ' -0.648005 -0.487707 -0.585006',
        ' -0.188574  -0.64144  0.743636'],
       ['0.829509 0.493747 0.261016', '  0.558395 -0.724402 -0.404273',
        ' 0.0105285 -0.481098  0.876604'],
       ['-0.671199  0.702859  0.235545',
        ' -0.678474 -0.454495 -0.577154',
        ' -0.298604 -0.547196  0.781929']], dtype=object)

## Split all elements by space

In [21]:
df=split_by_space(df)

In [23]:
df.values

array([[list(['', '0.731327', '-0.586307', '', '0.348433']),
        list(['', '-0.676398', '-0.688998', '', '0.260321']),
        list(['', '-0.0874416', '', '', '0.426059', '', '', '', '0.90046'])],
       [list(['-0.808864', '', '0.418552', '', '-0.41298']),
        list(['', '-0.531938', '-0.820171', '', '0.210619']),
        list(['', '', '0.250559', '-0.390043', '-0.886051'])],
       [list(['', '', '0.68376', '-0.617386', '', '0.388981']),
        list(['', '-0.729447', '-0.564085', '', '', '0.38693']),
        list(['', '0.0194665', '', '0.548309', '', '', '0.83605'])],
       ...,
       [list(['', '0.737923', '-0.592197', '-0.323687']),
        list(['', '-0.648005', '-0.487707', '-0.585006']),
        list(['', '-0.188574', '', '-0.64144', '', '0.743636'])],
       [list(['0.829509', '0.493747', '0.261016']),
        list(['', '', '0.558395', '-0.724402', '-0.404273']),
        list(['', '0.0105285', '-0.481098', '', '0.876604'])],
       [list(['-0.671199', '', '0.702859', 

## Remove empty strings in all elements

In [24]:
df_tmp_f=df.applymap(remove_empty_strings)

In [25]:
print(df_tmp_f.shape)
df_tmp_f.head()

(289521, 3)


,v1,v2,v3
0,"[0.731327, -0.586307, 0.348433]","[-0.676398, -0.688998, 0.260321]","[-0.0874416, 0.426059, 0.90046]"
1,"[-0.808864, 0.418552, -0.41298]","[-0.531938, -0.820171, 0.210619]","[0.250559, -0.390043, -0.886051]"
2,"[0.68376, -0.617386, 0.388981]","[-0.729447, -0.564085, 0.38693]","[0.0194665, 0.548309, 0.83605]"
3,"[-0.770767, -0.127292, -0.624272]","[-0.627478, -0.0181393, 0.778423]","[0.110411, -0.991699, 0.0658919]"
4,"[0.693495, 0.2807, 0.66353]","[-0.655229, -0.137211, 0.742865]","[-0.299566, 0.949937, -0.088768]"


## Unpack DataFrame values into array and flatten

In [27]:
ar=df_tmp_f.values
ar.shape

ar_fl=ar.flatten()
ar_fl

array([list(['0.731327', '-0.586307', '0.348433']),
       list(['-0.676398', '-0.688998', '0.260321']),
       list(['-0.0874416', '0.426059', '0.90046']), ...,
       list(['-0.671199', '0.702859', '0.235545']),
       list(['-0.678474', '-0.454495', '-0.577154']),
       list(['-0.298604', '-0.547196', '0.781929'])], dtype=object)

In [ ]:
final_ar = unpacked_array.reshape((df.shape[0], 3, 3))
final_ar.shape

## Reshape array

In [28]:
# Unpack lists and convert elements to float

unpacked_array = np.array([[float(x) for x in sublist] for sublist in ar_fl])
print(unpacked_array.shape)

# reshape to final desired shape N_cells X 3 X 3

final_ar = unpacked_array.reshape((df.shape[0], 3, 3))
final_ar.shape

(868563, 3)


(289521, 3, 3)

In [32]:
final_ar[1, ...]

array([[-0.808864,  0.418552, -0.41298 ],
       [-0.531938, -0.820171,  0.210619],
       [ 0.250559, -0.390043, -0.886051]])

In [30]:
df

,v1,v2,v3
0,"[, 0.731327, -0.586307, , 0.348433]","[, -0.676398, -0.688998, , 0.260321]","[, -0.0874416, , , 0.426059, , , , 0.90046]"
1,"[-0.808864, , 0.418552, , -0.41298]","[, -0.531938, -0.820171, , 0.210619]","[, , 0.250559, -0.390043, -0.886051]"
2,"[, , 0.68376, -0.617386, , 0.388981]","[, -0.729447, -0.564085, , , 0.38693]","[, 0.0194665, , 0.548309, , , 0.83605]"
3,"[-0.770767, -0.127292, -0.624272]","[, , -0.627478, -0.0181393, , , 0.778423]","[, , 0.110411, -0.991699, 0.0658919]"
4,"[0.693495, , , 0.2807, , 0.66353]","[, -0.655229, -0.137211, , 0.742865]","[, -0.299566, , 0.949937, -0.088768]"
...,...,...,...
289516,"[, 0.767785, -0.556575, -0.317381]","[, 0.609396, 0.481406, 0.629988]","[, , 0.197847, , 0.677106, -0.708791]"
289517,"[, 0.682642, -0.584871, -0.438094]","[, -0.730017, -0.518917, -0.444747]","[, -0.032785, -0.623419, , , , 0.7812]"
289518,"[, 0.737923, -0.592197, -0.323687]","[, -0.648005, -0.487707, -0.585006]","[, -0.188574, , -0.64144, , 0.743636]"
289519,"[0.829509, 0.493747, 0.261016]","[, , 0.558395, -0.724402, -0.404273]","[, 0.0105285, -0.481098, , 0.876604]"
